In [9]:
import os
import json
import numpy as np
import laspy as lp
import pyvista as pv
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

def get_las_file_paths(root_dir):
    las_file_paths = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.las'):
                las_file_paths.append(os.path.join(root, file))
    return las_file_paths

def view_cloud_pv(points):
    xyz = points[:, :3]  # Primeras 3 columnas: X, Y, Z
    rgb = points[:, 3:6]  # Columnas 4-6: R, G, B
    
    # Normalizar RGB de 16-bit (LAS) a 8-bit (0-255)
    if np.issubdtype(rgb.dtype, np.integer) and rgb.max() > 255:
        rgb = (rgb // 256).astype(np.uint8)  # Conversión a 8-bit
    
    # Crear objeto PolyData con coordenadas
    cloud = pv.PolyData(xyz)
    
    # Añadir colores RGB como array asociado a los puntos
    cloud["RGB"] = rgb.astype(np.uint8)  # Asegurar tipo uint8
    
    # Configurar plotter
    plotter = pv.Plotter()
    plotter.add_mesh(
        cloud,
        scalars="RGB",  # Usar el array RGB
        rgb=True,       # Indicar que son canales RGB
        point_size=3,   # Tamaño de los puntos
        opacity=1       # Opacidad total
    )
    
    plotter.show()

def get_points(path):
    las = lp.read(path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    rgb = np.vstack((las.red, las.green, las.blue)).transpose()
    data_points = np.hstack((
        points,
        rgb,
        np.expand_dims(las.return_number, -1),
        np.expand_dims(las.number_of_returns, -1)
    ))

    single = data_points[data_points[:, -1] == 1][:, :6]  #XYZRGB
    return single

def get_roi(points, p=0.05):
    # Obtener límites originales
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])
    
    # Calcular 5% del largo de cada eje
    x_offset = p * (x_max - x_min)
    y_offset = p * (y_max - y_min)
    
    # Ajustar límites hacia adentro
    new_x_min = x_min + x_offset
    new_x_max = x_max - x_offset
    new_y_min = y_min + y_offset
    new_y_max = y_max - y_offset
    
    return new_x_min, new_x_max, new_y_min, new_y_max

def bresenham_line(start=(10, 10), end=(10, 60)):
    # Unpack start and end points
    x1, y1 = np.array(start, dtype=np.int32).copy()
    x2, y2 = np.array(end, dtype=np.int32).copy()


    # Calculate differences
    dx = x2 - x1
    dy = y2 - y1

    # Determine if the line is steep (more vertical than horizontal)
    is_steep = abs(dy) > abs(dx)

    # Swap coordinates if the line is steep (for easier handling)
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    # Ensure the line is always drawn left-to-right
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    # Recalculate differences after the possible swap
    dx = x2 - x1
    dy = y2 - y1

    # Error term initialized to half of dx
    error = dx // 2
    ystep = 1 if y1 < y2 else -1  # Determines whether to increment or decrement y

    # List to store the generated points
    points = []
    y = y1

    # Main loop for Bresenham's algorithm
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)  # Swap x and y if the line is steep
        points.append(coord)

        # Update error term
        error -= abs(dy)

        # If error is negative, adjust y and reset the error term
        if error < 0:
            y += ystep
            error += dx

    return points

In [10]:
import laspy as lp
import numpy as np
import matplotlib.pyplot as plt
import glob

# pozas a analizar
# 05-04-2025, PAM4:6, RBN, RBS, 1B, 2B, 3B  (100-150m_10ms_100khz, 150m_7ms_100khz)
# 06-04-2025, 6B,7B,8B (100-150m_10ms_100khz)
# 07-04-2025 9D,9E,9F (07-04-25 100m&150m 100Khz 10ms)
# 20-05-2025 7H-7I (250520_164647_7H-7I)

rasantes = {
    'km-11': 2300.382,  
    'km-12': 2300.345,   
    'km-13': 2300.372,
    '3-a': 2300.305,
    '2-a': 2300.700,
    'pam-2': 2300.618  
}

salmueras = {
    'km-11': 2300.866,
    'km-12': 2300.582,
    'km-13': 2300.647,
    'pam-2': 2302.185,
    '2-a': 2301.129,
    '3-a': 2300.978
}

pozas = {
    "05-04-2025": [
        (558911.202,	7395022.839,	2302.254),
        (559066.708,	7395213.229,	2302.312),
        (559426.535,	7395214.473,	2302.104),
        (559424.764,	7394923.6,	    2302.141),
        (559086.394,	7394922.151,	2302.548)
    ],
    "06-04-2025": [
        (560911.613,	7394924.804,	2302.257),
        (560890.692,	7395213.724,	2302.17),
        (560226.493,	7395213.41,	    2302.125),
        (560272.775,	7394923.702,	2302.166)
    ],
    "07-04-2025": [
        (560017.361,	7396867.478,	2302.276),
        (560337.286,	7396821.869,	2302.183),
        (560336.371,	7397452.667,	2302.161),
        (560014.374,	7397467.77,	    2302.101)
    ],
    "20-05-2025": [
        (559312.555,	7397597.671,	2302.237),
        (559191.918,	7397634.896,	2300.614),
        (559181.084,	7397829.565,	2300.549),
        (559404.816,	7397855.527,	2300.577),
        (559420.061,	7397653.057,	2300.549),
        (559401.47,	    7397565.702,	2301.48),
        (559436.115,	7397321.4,	    2301.522),
        (559167.14,	    7397328.512,	2301.265),
        (559163.896,	7397558.826,	2301.331)
    ]
}

No existen las


In [11]:
import numpy as np
from sklearn.mixture import GaussianMixture
import pandas as pd

def find_best_gmm(data, max_components=2, criterion='aic', **kwargs):
    '''
    Find the best number of components for a Gaussian Mixture Model using AIC or BIC. 
    '''

    n_components_range = range(1, max_components + 1)
    criterions, means = [], []

    if criterion not in ['aic', 'bic']:
        raise ValueError("Invalid criterion. Use 'bic' or 'aic'.")

    for n in n_components_range:
        try:
            gmm = GaussianMixture(n_components=n, **kwargs)
            gmm.fit(data)
            if criterion == 'bic':
                criterions.append(gmm.bic(data))
            else:
                criterions.append(gmm.aic(data))
            means.append(gmm.means_)
        except ValueError as e:
            print(f"Error fitting GMM with {n} components: {e}")
            break

    return n_components_range[np.argmin(criterions)], means[np.argmin(criterions)]


In [14]:
import gc 
import numpy as np
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


dates = list(pozas.keys())

for date in dates:
    print('Date:', date)
    main_path = f"/home/diego/Downloads/data_sqm/Vuelos_{pozas[date]}/*"
    CONDITION_PATHS = glob.glob(main_path)
    for cond in CONDITION_PATHS:
        print('Date-Condition: ', date, cond)
        try:
            las = glob.glob(f"{cond}/*.las")
            #print(las)
        except:
            print("No existen archivos LAS para analizar")
            continue
        ## Get info
        for path in las:
            print('Date-Condition-LAS: ', date, cond, path)
            _,_,_,vuelo,condition,filename = path.split('/')
            DATE = vuelo.replace('Vuelos_',  '')
            control_points = pozas.get(DATE)

            ## Analysis
            points = get_points(path)
            x_min, x_max, y_min, y_max = get_roi(points)
            mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max) & \
            (points[:, 1] >= y_min) & (points[:, 1] <= y_max)
            roi_points = points[mask]

            del points
            gc.collect()    

            x, y, z = roi_points[:,0], roi_points[:,1], roi_points[:,2]

            points = np.vstack((x, y)).transpose()
            kdtree = KDTree(points)
            radius = 4

            for idx, (xi, yi, cota_sal_sqm) in enumerate(control_points, start=1):
                point_of_interest = np.array([xi, yi])
                indices = kdtree.query_ball_point(point_of_interest, radius)
                selected_z = z[indices]
                # z diffs
                diffs = np.abs(selected_z - cota_sal_sqm)
                # find the index of the minimum difference
                if len(diffs) == 0:
                    cota_sal_min_error = np.nan
                    distance = np.nan
                else:
                    min_index = np.argmin(diffs)
                    # get the corresponding z value
                    cota_sal_min_error = selected_z[min_index]
                    distance = np.linalg.norm(points[indices][min_index] - point_of_interest)
                # Clip by salt level
                #clipped_z = selected_z[selected_z >= rasante]
                cota_sal_promedio_original = np.mean(selected_z)
                #cota_sal_promedio_clipped = np.mean(clipped_z)
                #n, cota_sal = find_best_gmm(np.expand_dims(z[indices],-1), max_components=1)
                print(f'Cota sal SQM: {cota_sal_sqm}')
                print(f'Cota sal promedio original: {cota_sal_promedio_original}')
                #print(f'Cota sal promedio clipped: {cota_sal_promedio_clipped}')

                # find the nearest point to the point of interest
                nearest_index = kdtree.query(point_of_interest)[1]
                nearest_point = points[nearest_index]
                nearest_z = z[nearest_index]
                min_distance = np.linalg.norm(nearest_point - point_of_interest)
                #print(f'Nearest point: {nearest_point}')
                print(f'Z del punto más cercano al punto de interés: {nearest_z}', f'Distancia: {min_distance}')
                print(f'Cota sal promedio con error mínimo: {cota_sal_min_error}', f'Distancia: {distance}')
                print("")

Date: 05-04-2025
Date: 06-04-2025
Date: 07-04-2025
Date: 20-05-2025
